In [12]:
# Importer les bibliothèques nécessaires
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Charger le jeu de données
df = pd.read_csv('../data/df_merged_2020-2022.csv', sep=',', index_col="Num_Acc", low_memory=False)

df = df.replace('\xa0', '', regex=True)

# Nettoyer les données
# Remplacer les valeurs non numériques par NaN et les supprimer
df = df.apply(pd.to_numeric, errors='coerce')

In [13]:
df.head(5)

,id_vehicule,num_veh,place,catu,grav,sexe,an_nais,trajet,senc,catv,...,Casque,Dispositif enfants,Gilet réfléchissant,Airbag,Gants,Autre,date,weekday,heure,age
Num_Acc,,,,,,,,,,,,,,,,,,,,,
202200000001,813952,NaN,1,1,3.0,1.0,2008.0,5.0,1.0,2.0,...,True,False,False,False,False,False,NaN,NaN,16,16.0
202200000001,813953,NaN,1,1,1.0,1.0,1948.0,5.0,1.0,7.0,...,False,False,False,False,False,False,NaN,NaN,16,76.0
202200000002,813950,NaN,1,1,4.0,1.0,1988.0,9.0,2.0,7.0,...,False,False,False,False,False,False,NaN,NaN,8,36.0
202200000002,813951,NaN,1,1,1.0,1.0,1970.0,4.0,2.0,10.0,...,False,False,False,False,False,False,NaN,NaN,8,54.0
202200000003,813948,NaN,1,1,1.0,1.0,2002.0,0.0,2.0,7.0,...,False,False,False,False,False,False,NaN,NaN,17,22.0


In [14]:
# Séparer les caractéristiques (features) et la cible (target)
X = df.drop('grav', axis=1)
y = df['grav']

# Diviser le jeu de données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialiser le modèle RandomForest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

In [19]:
# Définir la grille des hyperparamètres à tester
param_distributions = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

random_search = RandomizedSearchCV(
    estimator=rf_model,
    param_distributions=param_distributions,
    n_iter=30,  # Le nombre d'itérations à tester
    scoring='accuracy',
    cv=3,  # Nombre de splits pour la validation croisée
    verbose=2,
    random_state=42,
    n_jobs=-1  # Utiliser tous les cœurs pour accélérer la recherche
)

In [20]:
# Entraîner RandomizedSearchCV
random_search.fit(X_train, y_train)

# Meilleurs hyperparamètres
print(f"Best parameters: {random_search.best_params_}")
print("Best score:", random_search.best_score_)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=  11.5s
[CV] END bootstrap=True, max_depth=40, min_samples_leaf=4, min_samples_split=2, n_estimators=100; total time=  43.2s


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=  11.5s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=  55.7s
[CV] END bootstrap=True, max_depth=20, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=  38.6s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=  55.6s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time= 1.2min
[CV] END bootstrap=True, max_depth=40, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time= 1.5min
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=4, min_samples_split=2, n_estimators=50; total time=  11.2s
[CV] END bootstrap=False, max_depth=40, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time= 1.2min
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1,

In [21]:
# Utiliser le meilleur modèle pour prédire les valeurs sur l'ensemble de test
best_rf_model = random_search.best_estimator_
y_pred = best_rf_model.predict(X_test)

In [22]:
# Évaluer le modèle
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

Accuracy: 0.6793595120091498
Classification Report:
              precision    recall  f1-score   support

         1.0       0.73      0.84      0.78     29052
         2.0       0.51      0.01      0.03      1659
         3.0       0.54      0.42      0.47      9532
         4.0       0.65      0.63      0.64     25332

    accuracy                           0.68     65575
   macro avg       0.61      0.48      0.48     65575
weighted avg       0.67      0.68      0.66     65575



### Interprétation de l'accuracy score et du classification report

#### Accuracy Score
L'accuracy score est une mesure de la fréquence à laquelle le classificateur fait des prédictions correctes. Il est calculé comme le ratio du nombre de prédictions correctes sur le nombre total de prédictions. Dans mon cas :

```plaintext
Accuracy: 0.6762790697674419
```

Cela signifie qu'environ 67,63% des prédictions faites par le modèle sont correctes.

#### Classification Report
Le classification report fournit une analyse détaillée des performances du classificateur pour chaque classe. Il inclut les métriques suivantes :

- **Précision (Precision)** : Le ratio des prédictions positives correctes sur le nombre total de prédictions positives. Il indique combien des instances prédites comme positives sont réellement positives.
- **Rappel (Recall)** : Le ratio des prédictions positives correctes sur le nombre total de vrais positifs. Il indique combien des instances réellement positives sont capturées par le modèle.
- **F1-Score** : La moyenne harmonique de la précision et du rappel. Elle fournit une métrique unique qui équilibre à la fois la précision et le rappel.
- **Support** : Le nombre d'occurrences réelles de la classe dans le jeu de données.

Voici le classification report que j'ai obtenu :

```plaintext
              precision    recall  f1-score   support

         1.0       0.72      0.84      0.78     29052
         2.0       0.42      0.04      0.07      1659
         3.0       0.53      0.42      0.47      9532
         4.0       0.65      0.62      0.64     25332

    accuracy                           0.68     65575
   macro avg       0.58      0.48      0.49     65575
weighted avg       0.66      0.68      0.66     65575
```

#### Interprétation
- **Classe 1.0** : Le modèle performe bien avec une précision élevée (0.72) et un rappel élevé (0.84), résultant en un bon F1-score (0.78).
- **Classe 2.0** : Le modèle a des difficultés avec cette classe, ayant une précision faible (0.42) et un rappel très faible (0.04), menant à un F1-score médiocre (0.07).
- **Classe 3.0** : Le modèle a une performance modérée avec une précision (0.53) et un rappel (0.42), résultant en un F1-score de 0.47.
- **Classe 4.0** : Le modèle performe raisonnablement bien avec une précision (0.65) et un rappel (0.62), résultant en un F1-score de 0.64.

- **Macro Average** : C'est la moyenne de la précision, du rappel et du F1-score pour toutes les classes, traitant toutes les classes de manière égale. Cela montre une performance globale modérée.
- **Weighted Average** : C'est la moyenne de la précision, du rappel et du F1-score pour toutes les classes, pondérée par le nombre d'instances dans chaque classe. Cela montre une performance légèrement meilleure en raison du nombre plus élevé d'instances dans les classes 1.0 et 4.0.

Globalement, le modèle performe bien pour les classes 1.0 et 4.0 mais a des difficultés avec la classe 2.0. L'accuracy de 67,63% indique que le modèle est correct dans environ deux tiers de ses prédictions.